In [ ]:
import ibmos2spark

# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'api_key': '457ujvkZFpxzaSpTVd17JZcXboCkhgFD9oRYrmkNF-Bh',
    'service_id': 'iam-ServiceId-f8a7d4db-3036-49fd-b9b5-db115af6ca48',
    'iam_service_endpoint': 'https://iam.bluemix.net/oidc/token'}

configuration_name = 'os_f68250162c014fe99aa26a4b35ce9144_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-dd47bdd1-03d0-4e17-8204-7f738de96ff4',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token',
    'api_key': 'WHQii9cabgFDbT4mId7fqtmIxqAJ5MFypFzrCuWqFVHm'
}

configuration_name = 'os_4ed955c22f764c8e80316ae28dbb9b37_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
# Since JSON data can be semi-structured and contain additional metadata, it is possible that you might face issues with the DataFrame layout.
# Please read the documentation of 'SparkSession.read()' to learn more about the possibilities to adjust the data loading.
# PySpark documentation: http://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.json

df_data_1 = spark.read.json(cos.url('medium-sparkify-event-data.json', 'datascientistsparkify-donotdelete-pr-bz8dxtt9csbrcm'))
df_data_1.take(5)


In [ ]:

# import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, concat, count, desc, explode, lit, min, max, split, stddev, udf
from pyspark.sql.types import IntegerType
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA, RegexTokenizer, StandardScaler, StopWordsRemover, StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.sql import functions as f

In [ ]:
# create a Spark session
spark = SparkSession.builder \
    .master("local") \
    .appName("Spark - Capstone Project") \
    .getOrCreate()

# Load, and Clean Dataset: 

In [ ]:
print(df_data_1)

In [ ]:
df = df_data_1
df.show()

In [ ]:
df.printSchema()

In [ ]:
# Filter for null values and empty strings
#Check for empty strings in userId and sessionId
print( df.where(col("userId").isNull()).count() )
print( df.where(col("sessionId").isNull()).count() )
print( df.where(col("userId") == "").count() )
print( df.where(col("sessionId") == "").count() )
df = df.filter(df.userId !="")

In [ ]:
## Check for other null values 

dataset_size = df.count()
for c in df.columns:
    null_count_c = df.where(col(c).isNull()).count()
    proportion_null = null_count_c/dataset_size
    print( c + ' : ' + str(null_count_c) + ' : ' + str(proportion_null))


# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

## Define Churn
Once you've done some preliminary analysis, create a column Churn to use as the label for your model. I suggest using the Cancellation Confirmation events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the Downgrade events.

## Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [ ]:
# Timespan of the data


from pyspark.sql.functions import udf, col, lit, min as Fmin, max as Fmax, sum as Fsum, expr
from pyspark.sql.types import IntegerType, DateType, FloatType, DoubleType
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.classification import GBTClassifier, RandomForestClassifier, LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

import numpy as np
import pandas as pd

import datetime

print( 'Min timestamp = ' + str( datetime.datetime.utcfromtimestamp( df.agg(Fmin(col("ts"))).collect()[0][0]/1000  ).strftime('%c') ) )
print( 'Max timestamp = ' + str( datetime.datetime.utcfromtimestamp( df.agg(Fmax(col("ts"))).collect()[0][0]/1000  ).strftime('%c') ) )

In [ ]:
print(df.select("userId").dropDuplicates().count())
print(df.select("sessionId").dropDuplicates().count())

In [ ]:
# Distinct values taken by different categorical variables at the app level and their respective number of occurences in the data

# The following fields are excluded here since they are user activity specific. serId, sessionId, artist

categorical_variables = [ 'auth' , 'gender', 'level', 'method', 'page', 'status' ]

for cvar in categorical_variables:
    print(cvar)
#     df.select(cvar).dropDuplicates().sort(cvar).show(100, False)
    df.groupby(cvar).count().sort('count', ascending = False).show(100, False)

# Defining churn
#### We define churn as a user cancelling their service. If we see a "Cancellation Confirmation" event for a user, then we would consider that user to have churned

#### Given our definition of churn, downgrades of service from paid to free tier could be potential indicators of churn too.

In [ ]:
# Churn is a label for user who cancelled
# Define a flag function
from pyspark.sql import Window
# Order dataframe by userId and timestamp
df = df.orderBy("userId", "ts")

# Function to convert milliseconds to days
days = lambda x: x * (1000 * 60 * 60 * 24)

# Flag cancellation within the next 3 days
churn_window = Window.partitionBy(df['userId']).orderBy(df['ts']).rangeBetween(0, days(3))

df = df.withColumn(
    "churn", f.max(f.when(df.page == "Cancellation Confirmation", 1).otherwise(0)).over(churn_window)
)


# Collect userIds for users who churned
users_that_churned = df.filter(df['churn'] == 1).select('userId').dropDuplicates().collect()
users_that_churned_ids = [i.userId for i in users_that_churned]

# Isolate page actions for users that churned
churned = df.filter(df['userId'].isin(users_that_churned_ids))


In [ ]:

# Again define pandas df for better view
dfp = df.toPandas()

dfp.sample(10)

# Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played

In [ ]:

# Know number of users who remain and who canceled
# by pandas
print(dfp.drop_duplicates(subset='userId').groupby(['churn'])['userId'].count())

# Exploring user status by gender

print(df.dropDuplicates(['userId', 'gender']).groupby(['churn', 'gender']).count().show())


In [ ]:
# Exploring user status by paid lever

print(df.dropDuplicates(['userId', 'level']).groupby(['churn', 'level']).count().show())

def plot_frequency(subset, group, labels, x_title="Number of users", y_title="Subscription status"):
    ax = dfp.drop_duplicates(subset=subset).groupby(
                                group)['userId'].count().plot(
                                kind='barh', figsize=(8,5), 
                                title='Number of unique users per category');
    ax.set_xlabel(x_title);
    ax.set_yticklabels(labels, rotation=0)
    ax.set_ylabel(y_title);

# Test    
plot_frequency(['userId'], ['churn'], ['Active', 'Cancelled'])

In [ ]:
plot_frequency(['userId', 'gender'], 
               ['gender', 'churn'], 
               ['Active-Female', 'Cancelled-Female', 'Active-Male', 'Cancelled-Male'])



In [ ]:
plot_frequency(['userId', 'level'], 
               ['level', 'churn'], 
               ['Active-Free', 'Cancelled-Free', 'Active-Paid', 'Cancelled-Paid'])

In [ ]:
import re

agents = [x.strip() for x  in dfp.userAgent.unique()] 
# see sample
[agents[x] for x in [0, 2, 4, 10]]

# TO get the data between parenthesis
ex = '\(([^\)]*)\)'

# All OSs
sorted(list(set([re.findall(
    ex, x)[0].split(';')[0].capitalize(
) for x  in agents])))



In [ ]:
# Create mappers for the os
mapper_general = {'Compatible': 'Windows',  'Ipad': 'Mac',  'Iphone': 'Mac',  
          'Macintosh': 'Mac',  'Windows nt 5.1': 'Windows',  
          'Windows nt 6.0': 'Windows',  'Windows nt 6.1': 'Windows',  
          'Windows nt 6.2': 'Windows',  'Windows nt 6.3': 'Windows',  
          'X11': 'Linux'}
mapper_specific = {'Compatible': 'Windows 7',  'Ipad': 'iPad',  'Iphone': 'iPhone',  
          'Macintosh': 'MacOS',  'Windows nt 5.1': 'Windows XP',  
          'Windows nt 6.0': 'Windows Vista',  'Windows nt 6.1': 'Windows 7',  
          'Windows nt 6.2': 'Windows 8.0',  'Windows nt 6.3': 'Windows 8.1',  
          'X11': 'Linux'}
print(sorted(list(set([mapper_general[re.findall(ex, x)[0].split(';')[0].capitalize()] for x  in agents]))),
sorted(list(set([mapper_specific[re.findall(ex, x)[0].split(';')[0].capitalize()] for x  in agents]))))


# Define user defined functions
os_general = udf(lambda x: mapper_general[re.findall(ex, x)[0].split(';')[0].capitalize()])
df = df.withColumn("os_general", os_general(df.userAgent))

os_specific = udf(lambda x: mapper_specific[re.findall(ex, x)[0].split(';')[0].capitalize()])
df = df.withColumn("os_specific", os_specific(df.userAgent))



## Now we want to get a users states 

states = set([state[1].strip() for state in [x.split(',') for x in dfp.location.unique()]])
# Define a user defined function
get_state = udf(lambda x: x.split(',')[1].strip())
df = df.withColumn("state", get_state(df.location))

# Convert to pandas for better visuals
dfp = df.toPandas()
dfp.sample(5)

In [ ]:

def get_series(col, churn, normalize=False):
    ''' 
    Get a series either of churn users or existing users
    THe function can normalize the series and return a percent value
    col: {str} the column to plot (hour, month, day, week_day)
    churn: {int} 0 or 1
    normalize: {bool} T/F, Default is False
    return a sorted pandas series
    '''
    ser1 = dfp[dfp.churn == churn].groupby([col])['userId'].count()
    try:
        ser1.index = ser1.index.astype(int)
    except:
        pass
    if normalize:
        ser1=ser1/ser1.sum()*100
    return ser1.sort_index()

def draw_time(col, normalize=True, figsize=(16,4), title=None, label_rotation=0):
    '''
    Draw a bar plot for churn and existing users based on 
         a specific column (hour, month, day, week_day)
         
    normalize: {bool} T/F, Default is True
    figsize: {tuple} the plot size, default is (16,4)
    title: descriptive title part
    Returns: None
    '''
    df_time = pd.DataFrame({'Cancelled': get_series(col, 1, normalize),
                            'Active users':get_series(col, 0, normalize)})
    ax = df_time.plot(kind='bar', figsize=figsize);
    ax.set_ylabel('Percent of users')
    if title is None:
        title = col
    ax.set_ylabel(f'Percent of users')
    #print(ax.get_xticks())
    ax.set_xticklabels(ax.get_xticklabels(), rotation=label_rotation)
    ax.set_title(f'Percent of users took action per {title}') 
    return ax

In [ ]:
draw_time('os_general');

In [ ]:
draw_time('os_specific');

In [ ]:
draw_time('state', label_rotation=90);

# Feature Engineering
Initial EDA suggests maintaining the timeseries format of the data set may be our best tact. We will attempt to engineer features that lean in to this by collecting additional information from the timestamp and using rolling windows to summarize recent actions.

Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.


Overview
There are many featurs to keep, Categoric features (needs to be converted to numeric form through dummy encoding):
- Gender of user
- OS of user
- State of residence that person lives in
- Subscription level: paid or free
- Popular artists with user


Numeric features:
- Song length per user per session
- ThumbsUp/ThumpbsDown/InviteFriends/downgrades/songs per sissio/session duration/ length of subscription/number of days as free or paid user

In [ ]:
df.head()

In [ ]:
# Create hour / day / month / year fields
# Convert timestamp
to_time_string = f.udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).strftime("%Y-%m-%d %H:%M:%S"))
df = df.withColumn('timestamp', f.to_timestamp(to_time_string(df['ts'])))

# Roll up to datestamp
df = df.withColumn('datestamp', df['timestamp'].cast('date'))


df = df.withColumn("hour", f.hour("timestamp"))
df = df.withColumn("day", f.dayofmonth("timestamp"))
df = df.withColumn("month", f.month("timestamp"))
df = df.withColumn("year", f.year("timestamp"))

def days_since_reg(x, y):
    """
    Retrieve rounded number of days since registration
    """
    sec_diff = f.when(x > y, x - y).otherwise(0)    
    return f.round(sec_diff / (1000 * 60 * 60 * 24))


# Days since registration
df = df.withColumn(
    'days_since_reg', days_since_reg(df.ts, df.registration)
)

# Convert gender to binary column
df = df.withColumn('gender_male', f.when(df.gender == 'M', 1).otherwise(0))

# Convert subscription level to binary column
df = df.withColumn('paid_level', f.when(df.level == 'paid', 1).otherwise(0))

# List of unique page entries
pages = [row.page for row in df.select(df.page).dropDuplicates().collect()]

for page in pages:
    
    # Flag matching pages
    page_col = page.replace(" ", "")+"_page"
    df = df.withColumn(page_col, f.when(df.page == page, 1).otherwise(0))
    
    # for 1, 7, and 30 day intervals...
    for i in [1, 7, 30]:
                
        # Rolling window funciton for i days
        w = (Window().partitionBy(df['userId']).orderBy(df['ts']).rangeBetween(-days(i), 0))

        # Create column counting sum of page events in rolling window
        window_col = page.replace(" ", "") + "_last{}d".format(i)
        df = df.withColumn(window_col, f.sum(page_col).over(w))

In [ ]:
# Create features vector
exclude_cols = ['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
                'level', 'location', 'method', 'page', 'registration', 'sessionId', 'song', 'status',
                'ts', 'userAgent', 'userId', 'timestamp', 'datestamp', 'id', 'churn', 
                'CancellationConfirmation_page', 'CancellationConfirmation_last1d', 
                'CancellationConfirmation_last7d', 'CancellationConfirmation_last30d', 'state', 'os_specific', 'os_general']
feature_cols = [x for x in df.columns if x not in exclude_cols]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="raw_features")
df = assembler.transform(df)

In [ ]:

# number of changing songs
song_user_df = df.filter(df.page == 'NextSong').groupBy(
                        'userId', 'sessionId').count()
song_user_df = song_user_df.groupBy('userId').agg(
                        sF.avg('count').alias('mean_songs'), 
                        sF.stddev('count').alias('stdev_songs'))
song_user_df = song_user_df.na.fill(0)
song_user_df.show(3)

In [ ]:
# Number of artists the user fans
artists_user_fans = df.select('userId', 'artist').dropDuplicates().groupBy('userId').count().withColumnRenamed("count", "num_aritst")
artists_user_fans.show(3)

In [ ]:
## Now am finding the session duration

session_end = df.groupBy('userId', 'sessionId').max('ts').withColumnRenamed(
                                                            'max(ts)', 'end')
session_start = df.groupBy('userId', 'sessionId').min('ts').withColumnRenamed(
                                                            'min(ts)', 'start')
session_df = session_start.join(session_end,['userId', 'sessionId'])
ticks_per_hours = 1000 * 60 * 60
session_df = session_df.select('userId', 'sessionId', ((
    session_df.end-session_df.start)/ticks_per_hours).alias('session_hours'))

# 2-G.2  Get Average, and Standard deviation of the session duration per user

session_user_df = session_df.groupBy('userId').agg(
                        sF.avg('session_hours').alias('mean_session_h'), 
                        sF.stddev('session_hours').alias('stdev_session_h'))
# Fill NaN with 0
session_user_df = session_user_df.na.fill(0)
session_user_df.show(3)



In [ ]:
## Count sessions per user 

num_sessions_user_df = df.select('userId', 'sessionId').dropDuplicates().groupby('userId').count().withColumnRenamed('count', 'num_sessions')
num_sessions_user_df.show(3)


Number of ThumbsUp, ThumbsDown, InviteFriends, downgrades, etc...

In [ ]:
# To get the Number of ThumbsUp, ThumbsDown, InviteFriends, downgrades, ...
# I think it is better to get all actions except Chorn actions (Cancel, cancelation confirmation)
# The to normalize them as percent to sum all to 100.
from functools import reduce
# The distribution of pages per user (FILLING NAN with 0)
user_page_distribution = df.groupby('userId').pivot('page').count().na.fill(0) #.toPandas().head(30)

# Drop Cancel	Cancellation Confirmation columns
user_page_distribution = user_page_distribution.drop(*['Cancel','Cancellation Confirmation'])

# Normalizing each row to sum to 1

# SOURCES
# https://stackoverflow.com/questions/47641076/spark-normalize-each-row-of-a-dataframe
# https://stackoverflow.com/questions/31955309/add-column-sum-as-new-column-in-pyspark-dataframe

# the columns to be summed
pages_cols = user_page_distribution.columns[1:]

# Add a total column
new_df = user_page_distribution.withColumn('total', sum(user_page_distribution[col] for col in pages_cols))

# Apply normalization per column
for col in pages_cols:
    new_df = new_df.withColumn(f'norm_{col}', new_df[col] / new_df['total'] * 100.)
    
# Remove the total column    
new_df = new_df.drop('total')

# Remove the original columns
new_df = new_df.drop(*pages_cols)

# Rename the normalized columns back
oldColumns = new_df.columns
newColumns = ['userId'] + pages_cols
user_page_distribution = reduce(lambda new_df, idx: new_df.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), new_df)

# Freeup memory
new_df=None

user_page_distribution.toPandas().head(10)

In [ ]:
## I want to get the length of a user's subscription 

def subLength(df, col = 'days_on'):
    """
    Function gets timestamp and then merges into one dataframe
    """
    reg_ts = df.select('userId', 'registration').dropDuplicates().withColumnRenamed('registration', 'start')
    end_ts = df.groupBy('userId').max('ts').withColumnRenamed('max(ts)', 'end')
    reg_df = reg_ts.join(end_ts,'userId')
    ticks_per_day = 1000 * 60 * 60 * 24 
    reg_df = reg_df.select('userId', ((reg_df.end-reg_df.start)/ticks_per_day).alias(col))
    return reg_df

reg_df = subLength(df, col='days_total_subscription')
reg_df.show(10)
    

In [ ]:
## I'm like to see the number of days of each user id as a free or a paid user 

paid_free_df = df.filter(df.page=='NextSong').groupBy('userId').pivot('level').count()
paid_free_df = paid_free_df.na.fill(0)
active_cols = paid_free_df.columns[1:]
paid_free_df = paid_free_df.withColumn('total', 
                                       sum(paid_free_df[col] for col in active_cols))
for col in active_cols:
    paid_free_df = paid_free_df.withColumn(f'{col}_percent', 
                                           paid_free_df[col] / paid_free_df.total * 100)
active_cols.append('total')    
paid_free_df = paid_free_df.drop(*active_cols)
paid_free_df.show(10)

In [ ]:


# Isolate feature and label columns
data = df.select(df["churn"].alias("label"), df["raw_features"])

# Split code for training / validation
train, test = data.randomSplit([0.7, 0.3], seed=420)

# Instantiate standard scaler
scaler = StandardScaler(inputCol="raw_features", outputCol="features", withStd=True)

# Instantiate classifier
gbc = GBTClassifier(lossType='logistic', seed=420)

pipeline = Pipeline(stages=[scaler, gbc])


#def features_join(new, old):
#    df_joined = old
#    new = new.join(df_joined, 'userId', how = 'inner')
#    return new.dropDuplicates()
#
#for i, feature in enumerate(user_features):
#    final_df = features_join(final_df, feature)
#
#final_df = final_df.orderBy('userId', ascending=True)
#final_df = final_df.dropDuplicates()

In [ ]:
pfinal_df = final_df.toPandas()
pfinal_df.head()


# Model Creating, Refinement, and Evaluation

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()

paramGrid = (ParamGridBuilder()
             .addGrid(gbc.maxDepth, [2, 4, 6])
             .addGrid(gbc.maxBins, [20, 60])
             .addGrid(gbc.maxIter, [10, 20])
             .build())

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=3)

cvModel = crossval.fit(train)
predictions = cvModel.transform(test)
evaluator.evaluate(predictions)



In [ ]:
model = pipeline.fit(train)

In [ ]:
preds = model.transform(test)

In [ ]:
# Collect label and probability values
preds_list = preds.select('label','probability')\
    .rdd.map(lambda row: (float(row['probability'][1]), float(row['label'])))\
    .collect()

# Pass through roc_curve function
y_score, y_true = zip(*preds_list)
y_score_rev = [1 - x for x in y_score]
fpr, tpr, thresholds = roc_curve(y_true, y_score_rev, pos_label = 1)

# Plot ROC Curve
fig, ax = plt.subplots(1, 1, figsize=(12,7))
ax.plot(tpr, fpr)

# Format chart
ax.set_ylabel("True Positive Rate")
ax.set_xlabel("False Positive Rate")
ax.set_title("ROC Curve", fontsize=14)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show()

In [ ]:
# Convert predictions to a dataframe
preds_df = pd.DataFrame({'preds': y_score, 'label': y_true})

# Iterate over possible thresholds and capture metrics for each
thresholds = [x / 100.0 for x in list(range(1,99))]
accuracy = []
recall = []
precision = []
f1_score = []

for t in thresholds:
    tp = len(preds_df.query(f"label == 1 and preds >= {t}"))
    fp = len(preds_df.query(f"label == 0 and preds >= {t}"))
    tn = len(preds_df.query(f"label == 0 and preds < {t}"))
    fn = len(preds_df.query(f"label == 1 and preds < {t}"))
    
    acc = (tp+tn)/(tp+fp+tn+fn)
    rec = tp / (tp+fn)

    try:
        prec = tp / (tp+fp)
    except ZeroDivisionError:
        prec = 0
    
    try:
        f1 = (prec * rec) / (prec + rec)
    except:
        f1 = 0
    
    accuracy.append(acc)
    recall.append(rec)
    precision.append(prec)
    f1_score.append(f1)
    
# Plot Accuracy / Recall / Precision
fig, ax = plt.subplots(1, 1, figsize=(12,8))
ax.plot(thresholds, accuracy, label='Accuracy')
ax.plot(thresholds, recall, label='Recall')
ax.plot(thresholds, specificity, label="Specificity")
ax.plot(thresholds, precision, label='Precision')
ax.plot(thresholds, f1_score, label="F1 Score")

# Format chart
ax.set_ylabel("Performance Rate")
ax.set_xlabel("Threshold")
ax.set_title("Performance by Threshold", fontsize=14)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.legend()
plt.show()

# Reflection 

- Overall I think that next steps could be getting into data streaming with spark. Businesses are fueled off quick responses, so incorporating data streaming is vital for a company's success.
- I could work with more observations by increaseing data set size
- I could work on incorporating more parameters or going else where to get more data on the user.




# References
Tausend, F. “Hands-on: Predict Customer Churn.” Medium, Towards Data Science, 1 June 2019, towardsdatascience.com/hands-on-predict-customer-churn-5c2a42806266.
Patel, Neil. “How to Improve Your Subscription Based Business by Predicting Churn.” Neil Patel, 24 May 2019, neilpatel.com/blog/improve-by-predicting-churn/.
